# OpenRouteService python bindings

https://github.com/GIScience/openrouteservice-py

In [9]:
import openrouteservice as ors
import folium
import json
import random
from pprint import pprint

url_ors = 'http://ors:8080/ors'
url_vroom = 'http://vroom:8081'

pointsA = json.load(open('./data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('./data/mezzocorona.geojson'))['features'] #6 points

#Povo [46.0682, 11.1551]
center = [46.2147, 11.1065]
zoom = 15

#s = 0.005 #impostare a 0 se partenza e' uguale a destinazione
s = 0

point_start = [46.2170,11.1066]
point_end = [point_start[0], point_start[1] - s]


# OpenRouteService TSP Routing examples

- [API Directions](https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/post)
- [Preferences examples](https://github.com/GIScience/openrouteservice-docs#examples)
- [more examples](https://github.com/GIScience/openrouteservice-py/blob/master/examples/basic_example.ipynb)

In [20]:
M = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)
    
client = ors.Client(base_url=url_ors, key='')

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]


#include partenza e arrivo
dirccA = [list(reversed(point_start))] + ccA + [list(reversed(point_end))]
dirccB = [list(reversed(point_end))] + ccB + [list(reversed(point_end))]

routeA = client.directions(
    coordinates = dirccA,
    profile = 'driving-car',
    format = 'geojson',
    preference = 'shortest',#'fastest',
    #restrictions =  ... width,height(in metri), weight(in tonnellate)
    validate = False
)

routeB = client.directions(
    coordinates = dirccB,
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

polyA = [list(reversed(coord)) for coord in routeA['features'][0]['geometry']['coordinates']]
polyB = [list(reversed(coord)) for coord in routeB['features'][0]['geometry']['coordinates']]

folium.PolyLine(polyA, color='red').add_to(M)
folium.PolyLine(polyB, color='blue').add_to(M)

for c in (ccA + ccB):
    loc = list(reversed(c))
    folium.Marker(loc, icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(M)

folium.Marker(point_start, icon=folium.Icon(icon='truck', prefix='fa', color='blue')).add_to(M)
    
M

## VRP Vehicle Routing Problem 

Optimize job scheduling for multiple vehicles.
The next example makes more use of the power of the ORS optimization endpoint. Two vehicles are assigned to 6 jobs, where the capacity constraints are such that each vehicle can only carry out 3 jobs on its route.

- [API Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)
- [API VROOM](https://github.com/VROOM-Project/vroom/blob/master/docs/API.md)

In [35]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

vehicles = []

pprint([point_start,point_end])

# Veicolo 1
vehicles.append(ors.optimization.Vehicle(
    id = 1,
    capacity = [5],  # Limite di capacita' del Veicolo    
    profile = 'driving-hgv',
    start = list(reversed(point_start)), # posizione di partenza veicoli
    end = list(reversed(point_end)),
))
folium.Marker(point_start, icon=folium.Icon(icon='truck', prefix='fa', color='blue')).add_to(m)

# Veicolo 2
point_start2 = [point_start[0] + 0.0015, point_start[1] - 0.005]

vehicles.append(ors.optimization.Vehicle(
    id = 1,
    capacity = [5],  # Limite di capacita' del Veicolo    
    profile = 'driving-hgv',
    #restrictions =  ... width,height(in metri), weight(in tonnellate)
    start = list(reversed(point_start2)), # posizione di partenza veicoli
    end = list(reversed(point_start2)),
))
folium.Marker(point_start2, icon=folium.Icon(icon='truck', prefix='fa', color='red')).add_to(m)

#Bidoni
job_locs = ccA + ccB

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    jobs.append(ors.optimization.Job(
        id = idx,
        location = cc,
        #priority = [1-100]
        amount = [1] # Quantita' del Contenuto del Bidone
    ))
    folium.Marker(loc, icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)

m

[[46.217, 11.1066], [46.217, 11.1066]]


In [36]:
clientVrp = ors.Client(base_url=url_vroom, key='')

optimized = clientVrp.optimization(
    vehicles = vehicles,
    jobs = jobs,
    geometry = True
)

routeA = optimized['routes'][0]
routeB = optimized['routes'][1]

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(routeA['geometry'])['coordinates']],
    color='blue'
).add_to(m)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(routeB['geometry'])['coordinates']],
    color='red'
).add_to(m)

m

In [17]:
vehicle1 = optimized['routes'][0]

vehicle2 = optimized['routes'][1]

#vehicle1['steps']

optimized

{'code': 0,
 'summary': {'cost': 691,
  'unassigned': 1,
  'amount': [6],
  'service': 0,
  'duration': 691,
  'waiting_time': 0,
  'distance': 6098,
  'computing_times': {'loading': 10, 'solving': 2, 'routing': 22}},
 'unassigned': [{'id': 6,
   'location': [11.115074157714844, 46.211318718540376]}],
 'routes': [{'vehicle': 0,
   'cost': 234,
   'amount': [3],
   'service': 0,
   'duration': 234,
   'waiting_time': 0,
   'distance': 2561,
   'steps': [{'type': 'start',
     'location': [11.1066, 46.217],
     'arrival': 0,
     'duration': 0,
     'distance': 0},
    {'type': 'job',
     'location': [11.116554737091064, 46.21342719651941],
     'job': 5,
     'service': 0,
     'waiting_time': 0,
     'arrival': 116,
     'duration': 116,
     'distance': 1260},
    {'type': 'job',
     'location': [11.118743419647217, 46.21494168708413],
     'job': 4,
     'service': 0,
     'waiting_time': 0,
     'arrival': 150,
     'duration': 150,
     'distance': 1558},
    {'type': 'job',
   

In [27]:
vehicle2['steps']

[{'type': 'start',
  'location': [11.1016, 46.217],
  'arrival': 0,
  'duration': 0,
  'distance': 0},
 {'type': 'job',
  'location': [11.09722137451172, 46.21403596735467],
  'job': 2,
  'service': 0,
  'waiting_time': 0,
  'arrival': 109,
  'duration': 109,
  'distance': 732},
 {'type': 'job',
  'location': [11.092135906219482, 46.216560067112056],
  'job': 1,
  'service': 0,
  'waiting_time': 0,
  'arrival': 214,
  'duration': 214,
  'distance': 1345},
 {'type': 'job',
  'location': [11.095612049102783, 46.21736181569706],
  'job': 0,
  'service': 0,
  'waiting_time': 0,
  'arrival': 275,
  'duration': 275,
  'distance': 1812},
 {'type': 'end',
  'location': [11.1016, 46.217],
  'arrival': 331,
  'duration': 331,
  'distance': 2284}]